<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song_sd_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition with LSTM 
Acoustic features are loaded directly from data directory

In [1]:
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
!git clone https://github.com/bagustris/ravdess_song.git

Cloning into 'ravdess_song'...
remote: Enumerating objects: 1147, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 1147 (delta 49), reused 32 (delta 12), pack-reused 1057
Receiving objects: 100% (1147/1147), 232.30 MiB | 25.09 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Checking out files: 100% (1030/1030), done.


In [3]:
cd ravdess_song/

/content/ravdess_song


In [4]:
# assign hsf to X, lab to y; reshape X for LSTM
X = np.load('data/x.npy', allow_pickle=True)
y = np.load('data/y.npy', allow_pickle=True)

In [5]:
# reshape x untuk lstm
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [6]:
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [7]:
# functio to build LSTM model
def model_lstm():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [8]:
# create the model
model = model_lstm()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1, 386)            1544      
_________________________________________________________________
lstm (LSTM)                  (None, 1, 256)            658432    
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1

In [9]:
# train the Dense model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
57/57 [==============================] - 6s 21ms/step - loss: 1.4456 - accuracy: 0.4538
Epoch 2/100
57/57 [==============================] - 1s 20ms/step - loss: 0.6561 - accuracy: 0.7473
Epoch 3/100
57/57 [==============================] - 1s 20ms/step - loss: 0.4800 - accuracy: 0.8088
Epoch 4/100
57/57 [==============================] - 1s 20ms/step - loss: 0.3461 - accuracy: 0.8736
Epoch 5/100
57/57 [==============================] - 1s 20ms/step - loss: 0.3301 - accuracy: 0.8769
Epoch 6/100
57/57 [==============================] - 1s 20ms/step - loss: 0.2654 - accuracy: 0.8989
Epoch 7/100
57/57 [==============================] - 1s 21ms/step - loss: 0.2192 - accuracy: 0.9121
Epoch 8/100
57/57 [==============================] - 1s 20ms/step - loss: 0.2581 - accuracy: 0.9033
Epoch 9/100
57/57 [==============================] - 1s 21ms/step - loss: 0.1960 - accuracy: 0.9286
Epoch 10/100
57/57 [==============================] - 1s 20ms/step - loss: 0.1740 - accuracy: 0.9352

In [10]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

7/7 [==============================] - 1s 5ms/step - loss: 0.2062 - accuracy: 0.9608
Loss:  0.20618675649166107 --> Accuracy:  0.9607843160629272
